In [1]:
def get_song_segment_key(song_id, segment_id):
    return "song_%s_%d" % (song_id, segment_id)

def get_song_id_from_segment_key(segment_key):
    return segment_key.split("_")[1]

song_segments={}
song_segments_array={}

import random
def get_segment_sample(num_segments,k=200):
    if num_segments<k:
        return [i for i in range(num_segments)]
    else:
        return random.sample(range(0,num_segments),k)


In [2]:
from lib import hdf5_getters as hfg
import os
import csv
import string
import numpy as np
# Provide the starting root dir
# This is different for your particular setup!

indir = "/mnt/snap/data"

cnt = 0
limit = 100000000
import timeit
max_segment_size = 0
start = timeit.default_timer()
# Open the CSV file we will write to
songs  = None
songs = None
song_timbre = []
names = []
segments = {}


with open("output.csv", 'wb') as csvfile:
    # Column headers
    # Currently only 4 features being extracted
    # We can add as many as we want, just seperate with commas

    # Recursively visit each sub-dir till we reach the h5 files
    # Strip punctuation from features that are strings
    for root, dirs, filenames in os.walk(indir):
        for f in filenames:
            #print(os.path.join(root, f))

            # Use the hd5 wrappers to open the h5 file
            h5 = hfg.open_h5_file_read(os.path.join(root, f))

            #getting total number of songs in the h5 file
            total_songs = hfg.get_num_songs(h5)
            for songidx in range(total_songs):
                print("Song count ",cnt)
                artist_name = hfg.get_artist_name(h5,songidx)
                artist = artist_name.decode('utf-8').translate(string.punctuation)

                # Get the title of the song
                title_song = hfg.get_title(h5,songidx)
                title = title_song.decode('utf-8').translate(string.punctuation)

                # Get release
                release_song = hfg.get_release(h5,songidx)
                release = release_song.decode('utf-8').translate(string.punctuation)
                
                #getting segments
                segments_start = hfg.get_segments_start(h5, songidx)
                segments_pitches = hfg.get_segments_pitches(h5, songidx)
                segments_timbre = hfg.get_segments_timbre(h5, songidx)
                segments_loudness_start = hfg.get_segments_loudness_start(h5, songidx)
                segments_loudness_max = hfg.get_segments_loudness_max(h5, songidx)
                track_id = hfg.get_track_id(h5, songidx)
                song_id = hfg.get_song_id(h5, songidx)
                N = segments_start.shape[0]
                segment_array=[]
                
                for i in get_segment_sample(N):
                    segment = np.array([])
                    segment = np.concatenate((segment, segments_pitches[i]), axis=0)
                    segment = np.concatenate((segment, segments_timbre[i]), axis=0)
                    segment = np.concatenate((segment, np.array([segments_loudness_start[i]])), axis=0)
                    segment = np.concatenate((segment, np.array([segments_loudness_max[i]])), axis=0)
                    segment_key = get_song_segment_key(song_id, i)
                    song_segments[segment_key] = segment
                    segment_array.append(segment)
                song_segments_array[song_id] = segment_array
                segments_bars_start = hfg.get_bars_start(h5, songidx)
                # Get duration
                duration = hfg.get_duration(h5,songidx)

                N = segments_start.shape[0]


                #segments_start = segments_start.reshape(N,1)
                #song_segment_data = np.stack(segments_start,segments_pitches)
                #print(song_segment_data.shape)
                #song = np.array([segments_start,segments_pitches,segments_timbre, \
                #                segments_loudness_start, segments_loudness_max, \
                #                segments_bars_start],dtype=object)
                #songs = np.append(songs, song)
                
                #print(segments_timbre.shape)
                song_timbre.append(segments_timbre)
                
                #########################################################

                # Print the current song and arists:
                max_segment_size = max(max_segment_size, len(segments_start))
                #print(title + " by " + artist + " duration: " ,duration/60.0," minutes "," segments:",len(segments_start)," songid ",song_id,"trackid ",track_id)
                # print str(num_songs)
                # Move on to the next h5 file
                #print("")
                print("")
                cnt+=1
                if cnt==limit:
                    break

            
            # Close the h5 file
            h5.close()
            if cnt==limit:
                break
        if cnt==limit:
            break
stop = timeit.default_timer()
print("Total songs: ",cnt,"total time: ",stop-start," max segment: ",max_segment_size)
print("Total song segments ",len(song_segments))


Song count  0

Song count  1

Song count  2

Song count  3

Song count  4

Song count  5

Song count  6

Song count  7

Song count  8

Song count  9

Song count  10

Song count  11

Song count  12

Song count  13

Song count  14

Song count  15

Song count  16

Song count  17

Song count  18

Song count  19

Song count  20

Song count  21

Song count  22

Song count  23

Song count  24

Song count  25

Song count  26

Song count  27

Song count  28

Song count  29

Song count  30

Song count  31

Song count  32

Song count  33

Song count  34

Song count  35

Song count  36

Song count  37

Song count  38

Song count  39

Song count  40

Song count  41

Song count  42

Song count  43

Song count  44

Song count  45

Song count  46

Song count  47

Song count  48

Song count  49

Song count  50

Song count  51

Song count  52

Song count  53

Song count  54

Song count  55

Song count  56

Song count  57

Song count  58

Song count  59

Song count  60

Song count  61

Song count  62

So

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




Song count  257

Song count  258

Song count  259

Song count  260
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-4089d37cf4d3>", line 54, in <module>
    segments_start = hfg.get_segments_start(h5, songidx)
  File "/home/ubuntu/recomuse/lib/hdf5_getters.py", line 319, in get_segments_start
    if h5.root.analysis.songs.nrows == songidx + 1:
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/tables/group.py", line 840, in __getattr__
    return self._f_get_child(name)
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/tables/group.py", line 715, in _f_get_child
    return self._v_file._get_node(childpath)
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/tables/file.py", line 1599, in _get_node
    node = self._node_manager.get_node(nodepath)
  File "/home/ubuntu/anaconda3/lib/py

KeyboardInterrupt: 

In [3]:
import numpy

from nearpy import Engine
from nearpy.hashes import RandomBinaryProjections
from nearpy.filters import NearestFilter
from nearpy.storage import MemoryStorage
# Dimension of our vector space
dimension = 26
print(segments)
# Create a random binary hash with 10 bits
rbp = RandomBinaryProjections('rbp', 10)

# Create engine with pipeline configuration
engine = Engine(dimension,\
                lshashes=[rbp],\
                vector_filters=[NearestFilter(5)],\
               storage=MemoryStorage())

# Index 1000000 random vectors (set their data to a unique string)
for segment_key in song_segments:
    v = song_segments[segment_key]
    engine.store_vector(v, segment_key)
print("indexed")


ModuleNotFoundError: No module named 'nearpy'

In [ ]:
def get_user_song_set(filename):
    user_songs={}
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter='\t')
        for row in csv_reader:
            user_id=row[0]
            song_id=row[1]
            if user_id not in user_songs:
                user_songs[user_id]=set()
            if song_id in song_segments_array:
                user_songs[user_id].add(song_id)
    return user_songs


In [ ]:
user_songs=get_user_song_set("Collaborative-Filtering-Million-Song-Dataset/year1_test_triplets_hidden-test.txt")

In [ ]:
print(user_songs)

In [ ]:
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
import numpy as np

def average_precision(actual_songs, predicted_songs, k=10):
    #print(len(predicted_songs),len(actual_songs))
    if len(predicted_songs) > k:
        predicted_songs = predicted_songs[:k]
    
    print(len(predicted_songs),len(actual_songs))
    if not actual_songs:
        return 0.0

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted_songs):
        if p in actual_songs and p not in predicted_songs[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
            print(score, num_hits)
    return score / min(len(actual_songs), k)


def mean_average_precision(predicted_songs, actual_songs, k=10):
    print("map")
    return np.mean([average_precision(a, p, k) for a, p in zip(actual_songs, predicted_songs)])

In [ ]:
from random import sample
def get_nearest_songs(engine, song_id):
    song_id=song_id.encode()
    if song_id in song_segments_array:
        pass
    if not (song_id in song_segments_array):
        return []
    song_segments=song_segments_array[song_id]
    song_scores={}
    song_segments = sample(song_segments,3)
    for segment in song_segments:
        near_segments = engine.neighbours(segment)
        for near_segment in near_segments:
            segment_key = near_segment[1]
            song_id = get_song_id_from_segment_key(segment_key)
            if song_id not in song_scores:
                song_scores[song_id]=0
            song_scores[song_id]+=1
    return song_scores

In [ ]:
import operator
def get_all_nearest_songs(engine, songs):
    all_nearest_songs = {}
    cnt=0
    print("no of songs for input ",len(songs))
    for song_id in songs:
        #print("song ",cnt)
        ns = get_nearest_songs(engine, song_id)
        for ns_key in ns:
            if ns_key not in all_nearest_songs:
                all_nearest_songs[ns_key]=ns[ns_key]
            else:
                all_nearest_songs[ns_key]+=ns[ns_key]
        cnt+=1
    return all_nearest_songs

def predicted_actual(engine, user_song_set):
    user_predictions = {}
    actual = []
    predicted = []
    for user in user_song_set:
        songs = list(user_song_set[user])
        n =  len(songs)
        songs_in = songs[:(n//2)]
        songs_out = songs[(n//2):]

        nearest_songs = get_all_nearest_songs(engine, songs_in)
        sorted_by_likelihood = sorted(nearest_songs.items(), key=operator.itemgetter(1),reverse=True)
        pred = [a[0] for a in sorted_by_likelihood]
        if len(pred)==0:
            continue
        predicted.append(pred[:n//2])
        actual.append(songs_out)
    return predicted, actual
        


In [ ]:
print(len(song_segments))
print("Num of users ",len(user_songs))
predicted, actual = predicted_actual(engine, user_songs)


In [ ]:
print(predicted)
print(actual)
print(mean_average_precision(predicted, actual))

In [ ]:
print('SONFOEG12A8C13B8B8'.encode())